In [1]:
import pandas as pd
import numpy as np

Shows movie dataset

In [2]:
movie_dataset = "Dataset/Netflix_Dataset_Movie.csv"
movie_df = pd.read_csv(movie_dataset)
movie_df.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


Shows user dataset

In [3]:
user_dataset = "Dataset/Netflix_Dataset_Rating.csv"
user_df = pd.read_csv(user_dataset)
user_df.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


The below calculation shows how many movies are rated by no users in our dataset. Because these movies have 0 ratings for all users, we will exclude them from our data. Since there are 16,420 movies that are unrated, our user-movie matrix will have signicantly fewer columns than the total number of movies in the dataset.

Sparsity Reduction: Movie recommendation datasets are typically sparse, meaning most movies are not rated by most users. Removing movies that have zero ratings can significantly reduce the sparsity of our matrix, making matrix factorization techniques more effective and computationally feasible.

In [4]:
movies_in_movie_df = set(movie_df['Movie_ID'])
movies_in_user_df = set(user_df['Movie_ID'])
unrated_movies = movies_in_movie_df - movies_in_user_df

print(f"Number of movies not rated by users: {len(unrated_movies)}")

Number of movies not rated by users: 16420


Combine these two dataframe objects in order to initalize our matrix. We assume that an "unrated" movie has a rating 0 for now.

In [5]:
merged_df = pd.merge(user_df, movie_df, on='Movie_ID')
pivot_table = merged_df.pivot_table(index='User_ID', columns='Name', values='Rating')
pivot_table = pivot_table.fillna(0)
pivot_table = pivot_table.sort_index()

In [6]:
print(f"Pivot Table Size: {pivot_table.shape[0]} rows, {pivot_table.shape[1]} columns")
pivot_table.head(10)

Pivot Table Size: 143458 rows, 1342 columns


Name,10,10 Things I Hate About You,101 Dalmatians II: Patch's London Adventure,11:14,13 Ghosts,18 Again,1984,2 Fast 2 Furious,200 Cigarettes,2010: The Year We Make Contact,...,Xena: Warrior Princess: Season 3,Xena: Warrior Princess: Series Finale,Y Tu Mama Tambien,Yellow Submarine,Yi Yi,Yojimbo,Young Black Stallion,Youngblood,Yu-Gi-Oh!: The Movie,Zorro
User_ID,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
pivot_table.to_csv('Users-Movies Matrix.csv')